In [ ]:
!pip install pandas
!pip install numpy
!pip install opencv-python
!pip install sklearn
!pip install tensorflow
!pip install joblib
!pip install os

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
import matplotlib
import matplotlib.pyplot as plt
import os
import cv2
import PIL
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import joblib

In [ ]:
#inputs to the script

#Model Path
model_dir=r"C:\Users\DK524UF\OneDrive - EY\Documents\EY Ripples\Forest Carbon Project\model"


All_Region_dir=r"C:\Users\DK524UF\OneDrive - EY\Documents\EY Ripples\Forest Carbon Project\All_Regions"


LabelEncoderPath=r"C:\Users\DK524UF\OneDrive - EY\Documents\EY Ripples\Forest Carbon Project\archive\animals\label_encoder.joblib"

In [ ]:
model= keras.models.load_model(r"C:\Users\DK524UF\OneDrive - EY\Documents\EY Ripples\Forest Carbon Project\model")
label_encoder = joblib.load(LabelEncoderPath)

In [ ]:
def animal_identifier(region_dir,model):
    df=pd.DataFrame(columns=['Image_Name','Region_Name','Species'])
    for i in range(0,len(os.listdir(region_dir))):
        images=[]
        img = cv2.imread(region_dir+'/'+os.listdir(region_dir)[i])
        resized_img = cv2.resize(img,(224,224))
        resized_img = resized_img / 255.0
        images.append(resized_img)
        images = np.array(images,dtype = 'float32')
        preds = model.predict(images)
        preds = np.argmax(preds,axis = 1)
        pred_animal=label_encoder.inverse_transform([preds[0]])[0]
        df.loc[i,'Image_Name']=os.listdir(region_dir)[i]
        df.loc[i,'Region_Name']=region_dir.split('\\')[-1]
        df.loc[i,'Species']=pred_animal
    return df

In [ ]:
finaldf=pd.DataFrame(columns=['Image_Name','Region_Name','Species'])
for i in range(0,len(os.listdir(All_Region_dir))):
    region_dir=All_Region_dir+'\\'+os.listdir(All_Region_dir)[i]
    preds=animal_identifier(region_dir,model)
    finaldf=finaldf.append(preds,ignore_index=True)
    finaldf.to_csv('Region_Wise_Predictions.csv',index=False)